## DEMO: Quantum phenomena

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from scipy.constants import h, c, k, e, N_A, epsilon_0, Rydberg 

In [ ]:
# Rydberg constant in spec units

R_H_cm = Rydberg * 1e-2 # cm^-1

# Print to check
print("e =", e, "C")
print("h =", h, "J·s")
print("c =", c, "m/s")
print("ε0 =", epsilon_0, "F/m")
print("m_e =", m_e, "kg")
print("R_H =", R_H_cm, "cm^-1")


In [ ]:
def rydberg_cm(n1, n2, Z=1):  # returns cm^-1

    return R_H_cm * Z**2 * (1/n1**2 - 1/n2**2)

### Calculate wavelength of an electron

### Plot sepctral transitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_hydrogen_energy_levels(n_min=1, n_max=8):
    n_vals = np.arange(n_min, n_max + 1)
    E_eV = -13.6 / (n_vals ** 2)

    fig, ax = plt.subplots(figsize=(6, 4))
    for n, E in zip(n_vals, E_eV):
        ax.hlines(E, 0, 1, linewidth=2)
        ax.annotate(f"n={n}", xy=(1.01, E), xytext=(1.04, E),
                    textcoords="data", va="center", fontsize=10)

    ax.set_ylim(min(E_eV) - 0.5, 0.5)
    ax.invert_yaxis()
    ax.set_xlim(0, 1.2)
    ax.set_xticks([])
    ax.set_ylabel("Energy (eV)")
    ax.set_title("Bohr Energy Levels of Hydrogen")
    ax.axhline(0.0, linestyle="--", linewidth=1)  # ionization limit
    fig.tight_layout()
    return fig, ax

plot_hydrogen_energy_levels()
plt.show()


### Plot Black Body radiation curves

In [ ]:
def planck(wavelength, T):
    """
    Planck's Law to calculate the spectral radiance of black body radiation at temperature T.
    
    Parameters:
    - wavelength (float): Wavelength in meters.
    - T (float): Absolute temperature in Kelvin.
    
    Returns:
    - (float): Spectral radiance in W/(m^2*sr*m).
    """
   
    exponent = (h*c) / (wavelength*k*T)
    
    exponent = np.clip(exponent, None, 700)  # Clip values of exponent at 700 to avoid numerical overflow

    return (2.0*h*c**2) / (wavelength**5 * (np.exp(exponent) - 1))
    

def plot_black_body(T, show=False):
    """
    Plot the black body radiation spectrum for a given temperature T.
    """

    # Lets pick some beautiful colors
    from matplotlib import cm
    from cycler import cycler
    magma = cm.get_cmap('magma', 10)  # 10 discrete colors, feel free to change
    plt.rcParams['axes.prop_cycle'] = cycler(color=[magma(i) for i in range(magma.N)])

    wavelengths = np.linspace(1e-9, 5e-6, 1000)  # Wavelength range from 1 nm to 3 um
    intensities = planck(wavelengths, T)
    
    plt.plot(wavelengths*1e9, intensities, label=f'T={T}', lw=3)  # Convert wavelength to nm for plotting
    
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('$Intensity (W/m^2)$')
    plt.grid(True)
    plt.legend()

    max_int = max(planck(wavelengths, 5000)) # lets limit y axis to 5000
    plt.ylim(0, max_int)

    if show: # show is needed for interactive plots
        plt.show() 

In [ ]:
for T in range(3000, 5000, 200):
    plot_black_body(T)

In [ ]:
interactive(plot_black_body, T=(1000, 5000, 100), show=True)